In [ ]:
# This code was mostly created by ChatGPT
from __future__ import annotations

import random
from typing import Any

from faker import Faker
import numpy as np
import pandas as pd

fake = Faker()

In [ ]:
np.random.seed(42)
Faker.seed(42)
random.seed(42)

In [ ]:
n_records = 85_253

In [ ]:
sections_per_course_distribution: dict[float, float] = {
    1: 0.5288,
    2: 0.2028,
    3: 0.0946,
    4: 0.0601,
    5: 0.0274,
    6: 0.0286,
    7: 0.0211,
    8: 0.0033,
    9: 0.0030,
    10: 0.0051,
    11: 0.0079,
    12: 0.0031,
    13: 0.0032,
    15: 0.0029,
    18: 0.0014,
    19: 0.0016,
    20: 0.0026,
    24: 0.0012,
    35: 0.0013,
}

instructors_per_course_distribution: dict[float, float] = {
    0: 0.0609,
    1: 0.9224,
    2: 0.0168,
}

sections_per_instructor_distribution: dict[float, float] = {
    1: 0.3148,
    2: 0.3148,
    3: 0.1872,
    4: 0.0922,
    5: 0.0425,
    6: 0.0192,
    7: 0.0149,
    8: 0.0055,
    9: 0.0036,
    10: 0.0035,
    15: 0.0018,
}

students_per_section_distribution: dict[float, float] = {
    1: 0.0579,
    2: 0.0373,
    3: 0.0290,
    4: 0.0413,
    5: 0.0284,
    6: 0.0472,
    7: 0.0332,
    8: 0.0263,
    9: 0.0402,
    10: 0.0409,
    11: 0.0329,
    12: 0.0358,
    13: 0.0276,
    14: 0.0274,
    15: 0.0225,
    16: 0.0326,
    17: 0.0298,
    18: 0.0215,
    19: 0.0219,
    20: 0.0158,
    21: 0.0175,
    22: 0.0194,
    23: 0.0137,
    24: 0.0176,
    25: 0.0211,
    26: 0.0209,
    27: 0.0094,
    28: 0.0167,
    29: 0.0246,
    30: 0.0110,
    31: 0.0151,
    32: 0.0112,
    33: 0.0122,
    34: 0.0130,
    35: 0.0148,
    36: 0.0157,
    37: 0.0073,
    38: 0.0159,
    39: 0.0080,
    40: 0.0091,
    41: 0.0053,
    42: 0.0057,
    43: 0.0065,
    44: 0.0040,
    45: 0.0016,
    46: 0.0033,
    47: 0.0046,
    48: 0.0027,
    49: 0.0015,
    50: 0.0040,
    51: 0.0035,
    52: 0.0016,
    53: 0.0012,
    54: 0.0019,
    55: 0.0010,
    56: 0.0020,
    57: 0.0020,
    59: 0.0013,
    60: 0.0006,
    63: 0.0005,
    69: 0.0012,
    111: 0.0005,
}
classifications_distribution: dict[str, float] = {
    "Foo (Winter 2024)": 0.6120,
    "Foo (Fall 2023)": 0.1859,
    "Graduate (Winter 2024)": 0.0573,
    "None": 0.0379,
    "Foo (Spring 2024)": 0.0369,
    "Graduate (Fall 2023)": 0.0160,
    "Foo (Spring 2023)": 0.0151,
    "Foo (Summer 2023)": 0.0130,
    "Foo (Winter 2023)": 0.0064,
    "Foo (Fall 2022)": 0.0053,
    "Graduate (Spring 2023)": 0.0015,
    "Foo (Spring 2022)": 0.0017,
    "Graduate (Summer 2023)": 0.0014,
    "Graduate (Spring 2024)": 0.0013,
    "Graduate (Winter 2023)": 0.0010,
    "Graduate (Fall 2022)": 0.0005,
    "Graduate (Summer 2022)": 0.0005,
    "Foo (Fall 2021)": 0.0005,
    "Foo (Spring 2020)": 0.0004,
    "Foo (Fall 2019)": 0.0003,
    "Foo (Summer 2024)": 0.0004,
    "Foo (Winter 2022)": 0.0004,
    "Foo (Summer 2022)": 0.0003,
    "Graduate (Fall 2018)": 0.0002,
    "Graduate (Winter 2022)": 0.0002,
    "Foo (Fall 2020)": 0.0002,
    "Foo (Winter 2019)": 0.0002,
    "Foo (Winter 2020)": 0.0002,
    "Graduate (Winter 2018)": 0.0002,
    "Graduate (Spring 2021)": 0.0002,
    "Foo (Winter 2017)": 0.0002,
    "Graduate (Winter 2021)": 0.0002,
    "Graduate (Summer 2021)": 0.0002,
    "Freshman (Winter 2024)": 0.0001,
    "Foo (Winter 2021)": 0.0001,
    "Foo (Spring 2016)": 0.0001,
    "Graduate (Spring 2017)": 0.0001,
    "Graduate (Summer 2020)": 0.0001,
    "Graduate (Spring 2020)": 0.0001,
    "Foo (Spring 2019)": 0.0001,
    "Graduate (Fall 2020)": 0.0001,
    "Graduate (Spring 2019)": 0.0001,
    "Graduate (Spring 2022)": 0.0001,
    "Foo (Summer 2021)": 0.0001,
    "Graduate (Fall 2021)": 0.0001,
    "Foo (Fall 2016)": 0.0001,
    "Foo (Summer 2019)": 0.0001,
    "Freshman (Fall 2022)": 0.0001,
    "Graduate (Summer 2024)": 0.0001,
    "Foo (Fall 2024)": 0.0001,
    "Foo (Spring 2021)": 0.0001,
}
num_courses_per_student_distribution: dict[float, float] = {
    1: 0.1870,
    2: 0.2159,
    3: 0.2484,
    4: 0.1562,
    5: 0.0881,
    6: 0.0480,
    7: 0.0226,
    8: 0.0105,
    9: 0.0067,
    10: 0.0056,
    11: 0.0018,
    12: 0.0053,
    13: 0.0027,
    14: 0.0005,
    15: 0.0001,
    16: 0.0005,
    17: 0.0002,
}

In [ ]:
def make_probs_sum_to_one(d: dict[Any, float]) -> dict[Any, float]:
    """Make sure the probabilities in dict values add up to one"""
    return {key: value / sum(d.values()) for key, value in d.items()}

In [ ]:
def EV(d: dict[float, float], normalize: bool = True) -> float:
    """Compute the expected value of a dict mapping `float`s to their probability or
    to their weight (the latter if `normalize=True`)"""
    if normalize:
        d = make_probs_sum_to_one(d)
    return sum(key * value for key, value in d.items())

In [ ]:
avg_sections_per_course = EV(sections_per_course_distribution)
avg_instructors_per_course = EV(instructors_per_course_distribution)
avg_sections_per_instructor = EV(sections_per_instructor_distribution)
avg_students_per_section = EV(students_per_section_distribution)
avg_courses_per_student = EV(num_courses_per_student_distribution)

In [ ]:
def sample_from_dict(
    distribution: dict[Any, float],
    size: int | None = None,
    normalize: bool = True,
) -> Any:
    """Take a sample from a `dict`, where the keys are the sample space and the
    values are the weights of each key,"""
    if normalize:
        distribution = make_probs_sum_to_one(distribution)
    keys, weights = zip(*distribution.items())
    return np.random.choice(keys, p=weights, size=size)

In [ ]:
# Calculate the number of unique students based on the average number of courses per
# student
n_unique_students = int(n_records / avg_courses_per_student)

# Generate unique student data
unique_students = pd.DataFrame(
    {
        "Student ID": [
            # The builtin `random.sample` method from the Python standard lib can
            # efficiently sample from `range` objects, while `np.random.sample` needs to
            # construct the entire list in memory first.
            f"ID{rand_id:09}"
            for rand_id in random.sample(range(10**9), n_unique_students)
        ],
        "Student Alternate ID": np.NaN,
        "Student Name": [fake.name() for _ in range(n_unique_students)],
        "Student E-mail": [fake.email() for _ in range(n_unique_students)],
        "Classification": sample_from_dict(
            classifications_distribution, size=n_unique_students
        ),
        "Major": np.random.choice(
            [
                "Computer Science",
                "Business Administration",
                "Psychology",
                "Education",
                "Engineering",
            ],
            n_unique_students,
        ),
        "Cumulative GPA": np.clip(np.random.normal(3, 0.5, n_unique_students), 0, 4),
    }
).replace("None", np.NaN)

In [ ]:
# Generate Categories
def select_categories() -> str | None:
    categories = []

    def _include_category(chance: float = 0.9) -> bool:
        """Randomly decide to include a category or not"""
        return np.random.rand() < chance

    campus_options = {"On-line": 0.5, "Main Campus": 0.4, "Satellite Campus": 0.1}
    graduated_options = {  # Mutually exclusive with certain other categories
        "Graduated: Yes": 0.1,
        "Graduated: No": 0.9,
    }
    level_options = [
        "Undergraduate",
        "Graduate",
    ]
    hold_types = ["Hold: Financial", "Hold: Academic", "Hold: Administrative"]
    comp_rate_options = {
        "Completion Rate: >= 66.67%": 0.75,
        "Completion Rate: < 66.67%": 0.25,
    }
    start_term_options = [
        f"Start Term: {season} {year}"
        for year in range(2000, 2025)
        for season in ["Fall", "Winter", "Spring", "Summer"]
    ]
    start_term_options_dict = {
        term: float(i) for i, term in enumerate(start_term_options, start=1)
    }
    term_status_options = [  # Mutually exclusive with "Graduated: Yes"
        "Term Status: Registered",
        "Term Status: Not Registered",
    ]
    fafsa_options = [  # Mutually exclusive with "Graduated: Yes"
        "FAFSA: Yes",
        "FAFSA: No",
    ]

    # Build up categories one-by-one, randomly deciding which will be included.
    if _include_category():
        categories.append(f"Campus: {sample_from_dict(campus_options)}")
    if _include_category():
        categories.append(sample_from_dict(graduated_options))
    if _include_category():
        categories.append(np.random.choice(level_options))
    if _include_category(chance=0.10):  # Only 10% chance of showing a Hold
        categories.append(np.random.choice(hold_types))
    if _include_category():
        categories.append(sample_from_dict(comp_rate_options))
    if _include_category():
        categories.append(sample_from_dict(start_term_options_dict))

    if "Graduated: Yes" not in categories:
        # We can only be here if you do NOT show "Graduated: Yes"
        if _include_category():
            categories.append(np.random.choice(term_status_options))
        if _include_category():
            categories.append(np.random.choice(fafsa_options))

    return ", ".join(categories) if categories else None


unique_students["Categories"] = [
    select_categories() for _ in range(len(unique_students))
]

In [ ]:
def select_tags() -> str | None:
    tags = []

    # General tags
    general_tags = [
        "Honor Student",
        "Scholarship Recipient",
        "At Risk",
        "Needs Tutoring",
        "Athlete",
        "International",
        "Transfer",
    ]
    # Tuples of mutually exclusive tags
    mutually_exclusive_tags = [("Part-Time", "Full-Time")]

    max_tags = len(general_tags) + len(mutually_exclusive_tags)

    def _include_tag(chance: float = 1 - 0.675 ** (1 / max_tags)) -> bool:
        """Randomly decide to include a tag or not.
        We want a 67.5% chance that a student has no tags whatsoever.
        With 8 possible tags, a little algebra reveals that the default chance must be
        `1 - 0.675 ** (1 / num_possible_tags)`."""
        return np.random.rand() < chance

    for tag in general_tags:
        if _include_tag():
            tags.append(tag)

    for tuple_of_mutually_exclusive_tags in mutually_exclusive_tags:
        if _include_tag():
            tags.append(np.random.choice(tuple_of_mutually_exclusive_tags))

    return ", ".join(tags) if tags else None


unique_students["Tags"] = [select_tags() for _ in range(len(unique_students))]

In [ ]:
num_courses_per_student = sample_from_dict(
    num_courses_per_student_distribution, size=n_unique_students
)

In [ ]:
# Replicate each student entry based on the number of courses they're taking
replicated_students = unique_students.loc[
    unique_students.index.repeat(num_courses_per_student)
].reset_index(drop=True)

In [ ]:
# Course info
# Generate course-specific information for each enrollment
course_depts = ["MTH", "ENG", "HSC", "EDL"]
course_numbers = [
    f"{dept}-{str(i).zfill(3)}" for dept in course_depts for i in range(1, 101)
]  # Assuming 100 courses per department
course_titles = [
    fake.unique.sentence(nb_words=3) for _ in range(len(course_numbers))
]  # Unique course titles
replicated_students["Course Number"] = np.random.choice(
    course_numbers, len(replicated_students)
)
replicated_students["Course Name"] = [
    course_titles[course_numbers.index(cn)]
    for cn in replicated_students["Course Number"]
]
replicated_students["Section"] = [
    "".join(np.random.choice(list("0123456789"), 5))
    for _ in range(len(replicated_students))
]
replicated_students["Instructors"] = [
    (
        f"{fake.last_name()},"
        f" {fake.first_name()} (ID{''.join(np.random.choice(list('0123456789'), 9))})"
        f" <{fake.email()}>"
    )
    for _ in range(len(replicated_students))
]

In [ ]:
# Enrollment info
replicated_students["Dropped?"] = sample_from_dict(
    {"Yes": 0.24, "No": 0.76}, size=len(replicated_students)
)
replicated_students["Dropped Date"] = dropped_dates = [
    fake.date_between(start_date="-1y", end_date="today") if drop == "Yes" else None
    for drop in replicated_students["Dropped?"]
]

In [ ]:
# Add other enrollment-specific information (e.g., grades, attendance) in a similar
# manner
replicated_students["Midterm Grade"] = np.random.choice(
    ["A", "B", "C", "D", "F"], len(replicated_students)
)
replicated_students["Final Grade"] = np.random.choice(
    ["A", "B", "C", "D", "F"], len(replicated_students)
)
replicated_students["Total Progress Reports"] = np.random.poisson(
    0.4, len(replicated_students)
)
replicated_students["Absences"] = np.random.poisson(0.5, len(replicated_students))

In [ ]:
# Schedule information
replicated_students["Start Date"] = [
    fake.date_between(start_date="-3m", end_date="+3m")
    for _ in range(len(replicated_students))
]
replicated_students["End Date"] = [
    sd + pd.Timedelta(weeks=16) for sd in replicated_students["Start Date"]
]
replicated_students["Start Time"] = [
    f"{hour}:00 AM CT"
    for hour in np.random.choice(range(8, 12), len(replicated_students))
]
replicated_students["End Time"] = [
    f"{hour + np.random.choice([1, 2, 3])}:00 PM CT"
    for hour in np.random.choice(range(1, 5), len(replicated_students))
]
replicated_students["Class Days"] = [
    ", ".join(
        np.random.choice(
            ["M", "T", "W", "R", "F", "Sa"], size=np.random.randint(1, 3), replace=False
        )
    )
    for _ in range(len(replicated_students))
]

In [ ]:
# Create DataFrame
# and ensure that the dataframe is not longer than `n_records`
df_unordered_columns = replicated_students.iloc[:n_records]

# Order the columns as expected:
df = df_unordered_columns[
    [
        "Student Name",
        "Student E-mail",
        "Student ID",
        "Student Alternate ID",
        "Categories",
        "Tags",
        "Classification",
        "Major",
        "Cumulative GPA",
        # "Assigned Staff",
        "Course Name",
        "Course Number",
        "Section",
        "Instructors",
        "Dropped?",
        "Dropped Date",
        "Midterm Grade",
        "Final Grade",
        "Total Progress Reports",
        "Absences",
        # "Unexcused Absences",
        # "Excused Absences",
        # "Credit Hours",
        "Start Date",
        "End Date",
        "Start Time",
        "End Time",
        "Class Days",
    ]
].replace({None: np.NaN})

In [ ]:
df